In [1]:
#pip install fastf1

In [2]:
# Importas as libs Necessárias
import pandas as pd
import numpy as np
import random
import requests

# Não esqueça de instalar a biblioteca
# pip install fastf1
import fastf1 as ff1

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Fazer uma api rápida para extrair os dados da Ergast
# Acesse o site para verificar os dados 
# https://ergast.com/api/f1

# Função da API com a Ergast
def API_Ergast(api_endpoint: str):
    
    # Passar o link da API
    url = f'https://ergast.com/api/f1/{api_endpoint}.json'
    
    # Fazendo a quest com a URL para retornar um arquivo Json
    response = requests.get(url).json()
    
    return response['MRData']

In [5]:
# Especifique o número de rodadas que queremos em nosso gráfico (em outras palavras, especifique a rodada atual)
Rodadas = 18

# Iniciando um dataframe vazio para armazenar nossos dados
Dados_Campeonato = pd.DataFrame()

# Também queremos armazenar qual motorista dirige para qual equipe, o que nos ajudará mais tarde
Equipe_Motorista = {}

# Inicie um loop por todas as rodadas
for Loop in range(1, Rodadas + 1):
    
    # Faça uma solicitação ao endpoint driverStandings para a rodada atual
    Rodada_Atual = API_Ergast(f'current/{Loop}/driverStandings')
    
    # Obtenha a classificação a partir do resultado
    Classificacao = Rodada_Atual['StandingsTable']['StandingsLists'][0]['DriverStandings']
    
    # Inicie um dicionário para armazenar a classificação das rodadas atuais
    Classificacao_Atual = {'round': Loop}
    
    # Percorre todos os drivers para coletar suas informações
    for Mot in range(len(Classificacao)):
        
        # Motorista
        Motorista = Classificacao[Mot]['Driver']['code']
        
        # Posição
        Posicao = Classificacao[Mot]['position']
        
        # Armazene a posição dos motoristas
        Classificacao_Atual[Motorista] = int(Posicao)
        
        # Crie um mapeamento para a equipe de pilotos a ser usado para colorir as linhas
        Equipe_Motorista[Motorista] = Classificacao[Mot]['Constructors'][0]['name']


    # Acrescente a rodada atual ao nosso dataframe fial
    Dados_Campeonato = Dados_Campeonato.append(Classificacao_Atual, ignore_index=True)
    
# Defina a rodada como o índice do dataframe
Dados_Campeonato = Dados_Campeonato.set_index('round')

ValueError: If using all scalar values, you must pass an index

In [6]:
# Acrestar os dados para que possam ser usados como entrada para o gráfico
Dados_Campeonato_Parcial = pd.melt(Dados_Campeonato.reset_index(), ['round'])

KeyError: "The following 'id_vars' are not present in the DataFrame: ['round']"

In [ ]:
Dados_Campeonato

In [ ]:
# Definindo o tamanho do gráfico
sns.set(rc={'figure.figsize':(15, 8) })

# Iniciando a plotagem
fig, ax = plt.subplots()

# Definir o titulo do gráfico
ax.set_title('2021 Classificação até a 18º rodada', fontsize=18)

# Desenhe uma linha para cada piloto nos dados, percorrendo todas as classificações
# A razão de fazermos desta forma é para que possamos especificar a cor da equipe por piloto

# Loop para cada piloto
for driver in pd.unique( Dados_Campeonato_Parcial['variable'] ):
    
    # Plotar cada motorista
    sns.lineplot(
        x='round', y='value', 
        data = Dados_Campeonato_Parcial.loc[ Dados_Campeonato_Parcial['variable']==driver ],
    )

# Inverta o eixo Y para ter o líder do campeonato (# 1) no topo
ax.invert_yaxis()

# Defina os valores que aparecem nos eixos x e y
ax.set_xticks(range(1, Rodadas))
ax.set_yticks(range(1, 22))

# Set the labels of the axes
ax.set_xlabel('Rodada')
ax.set_ylabel('Posição na corrida')

# Desabilitar os grid do gráfico
ax.grid(False)

# Adicione o nome do motorista no final da linha
for Linha, Nome in zip( ax.lines, Dados_Campeonato.columns.tolist()):
    
    # Selecionando a Linhas
    y = Linha.get_ydata()[-1]
    x = Linha.get_xdata()[-1]
    
    # Incluino o nome do motorista no final da linha do gráfico
    text = ax.annotate(
        Nome,
        xy=(x + 0.1, y),
        xytext=(0, 0),
        color=Linha.get_color(),
        xycoords=(
            ax.get_xaxis_transform(),
            ax.get_yaxis_transform()
        ),
        textcoords='offset points'
    )